In [1]:
import numpy as np
import pandas as pd
import itertools as it
import pickle
import glob
import os
import string
import gc
import re
import time
import nltk
import spacy
import textacy
#import en_core_web_md
import sematch
import gensim

from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from tqdm import tqdm, tqdm_notebook
from scipy import sparse
from scipy.optimize import minimize

from clean_utils import *
from parser_utils import *
%load_ext autoreload
%autoreload 2

ImportError: No module named 'textacy'

In [2]:
def clean_lemmat_quora(df, shapes):
    t = time.time()
    print('Cleaning based on forums functions.')
    df = clean(df['comment_text'])
    print('Cleaning using textacy.')
    df.comment_text = df.comment_text.apply(lambda x: textacy.preprocess.preprocess_text(x, fix_unicode = True,
                                                                            lowercase = True,
                                                                            no_contractions = True,
                                                                            transliterate = True))
    print('Lemmatizing text.')
    SYMBOLS = set(' '.join(string.punctuation).split(' ') + ['...', '“', '”', '\'ve'])
    q1 = []
    for doc in tqdm(nlp.pipe(df['question1'], n_threads=8, batch_size=10000)):
        word_list = ([c.lemma_ for c in doc if c.lemma_ not in SYMBOLS])
        q1.append(' '.join(i for i in word_list))
    q1 = pd.DataFrame(q1)
    
    df['comment_text'] = q1  
    print('Correcting words. Using most probable substitutes.')
    df.comment_text = df.comment_text.apply(lambda x: (' '.join([correction(i) for i in word_tokenize(x)])))
    df_train = df.iloc[:shapes[0], :]
    df_test = df.iloc[shapes[0]:, :]
    print('Text cleaning done, time it took:', time.time() - t)
    return df_train, df_test

In [3]:
NROWS = 100

In [ ]:
data =  pd.read_csv('data/train.csv', sep=',', index_col='id', nrows=NROWS)
shapes = data.shape
data['train'] = 1
data = pd.concat([data, pd.read_csv('data/test.csv', sep=',', index_col='id', nrows=NROWS)], axis=0)
data.train.fillna(0, inplace=True)

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('vocs/crawl-300d-2M.vec')

In [ ]:
words = model.index2word

In [ ]:
1+1t

In [3]:
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank

In [18]:
nlp = spacy.load(en)
df_train, df_test = clean_lemmat_quora(src)

df_train.to_csv('df_train_spacylemmat_fullclean.csv', index = False)
df_test.to_csv('df_test_spacylemmat_fullclean.csv', index = False)

NameError: name 'enn' is not defined